In [1]:
import pandas as pd
from pathlib import Path
import pylatex as pl
from pylatex.utils import NoEscape
import pymc as pm
import arviz as az
import plotly.express as px
import numpy as np
import pandas as pd

from estival.model import BayesianCompartmentalModel
import estival.priors as esp
import estival.targets as est
from estival.wrappers import pymc as epm
from tbdynamics import model
from tbdynamics.inputs import fixed_parameters
from tbdynamics.utils import build_contact_matrix, round_sigfig
import plotly.graph_objects as go
#from emutools.parameters import load_param_info
#from emutools.calibration import round_sigfig
# from general_utils.doc_utils import TextElement, TableElement, FigElement, add_element_to_document, \
#     save_pyplot_add_to_doc, save_plotly_add_to_doc, compile_doc, generate_doc
from emutools.tex import StandardTexDoc

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
PROJECT_PATH = Path().resolve()
DATA_PATH = PROJECT_PATH / 'data'
SUPPLEMENT_PATH = PROJECT_PATH / 'supplement'

In [3]:
params = {
    "contact_rate": 0.009414102898074345,
    "start_population_size": 227344.75719536067,
    "cdr_adjustment": 0.6,
    "progression_multiplier": 1.1,
    "rr_infection_latent": 0.2,
    "rr_infection_recovered": 0.2,
    "infect_death_rate_unstratified": 0.21,
    "on_treatment_infect_multiplier": 0.08,
    'smear_positive_death_rate':0.364337776897486,
    'smear_negative_death_rate': 0.027588310343242016, 
    'smear_positive_self_recovery': 0.20344728302826143,
    'smear_negative_self_recovery': 0.22723824998716693,
    'rr_progression_diabetes': 4.5,
    'seed_time': 200.0,
    'seed_duration': 1.0,
    'seed_rate': 1.0

}

In [4]:
pd.options.plotting.backend = "plotly"
time_start = 1800
time_end = 2020
time_step = 0.1

doc_sections = {}
compartments = [
    "susceptible",
    "early_latent",
    "late_latent",
    "infectious",
    "on_treatment",
    "recovered",
]
infectious_compartments = [
    "infectious",
    "on_treatment",
]

latent_compartments = [
    "early_latent",
    "late_latent",
]
age_strata = [0,5,15,35,50,70] 

In [5]:
app_doc = StandardTexDoc(PROJECT_PATH / 'supplement', 'supplement', "TB dynamics Supplement", 'tb')

In [6]:
matrix = build_contact_matrix()

In [7]:
tb_model = model.build_model(
    compartments,
    infectious_compartments,
    latent_compartments,
    age_strata,
    time_start,
    time_end,
    time_step,
    matrix,
    fixed_parameters,
    app_doc, 
    False
)

In [8]:
priors = [
    esp.UniformPrior("start_population_size", (1, 300000)),
    esp.UniformPrior("contact_rate", (0.0001, 0.02)),
    esp.UniformPrior("rr_infection_latent", (0.2, 0.5)),
    esp.UniformPrior("rr_infection_recovered", (0.1, 0.5)),
    esp.UniformPrior("smear_positive_death_rate", (0.335, 0.449)),
    esp.UniformPrior("smear_negative_death_rate", (0.017, 0.035)),
    esp.UniformPrior("smear_positive_self_recovery", (0.177, 0.288)),
    esp.UniformPrior("smear_negative_self_recovery", (0.073, 0.209)),
    # esp.UniformPrior("rr_progression_diabetes", (2.0, 10.0)),
    esp.UniformPrior("progression_multiplier", (0.1, 2.0)),
    # esp.UniformPrior("acf_scaleup_shape", (0.,0.1)),
    # # esp.UniformPrior("acf_inflection_time", (1980,2019)),
    # esp.UniformPrior("acf_start_asymp", (0.,1.)),
    # esp.UniformPrior("acf_end_asymp", (0.,10.)),
    # UniformPrior("cdr_adjustment", [0.6, 1.0]),
    esp.UniformPrior("infect_death_rate_dict.smear_positive", [0.335, 0.449]),
    esp.UniformPrior("infect_death_rate_dict.smear_negative", [0.017, 0.035]),
    esp.UniformPrior("self_recovery_rate_dict.smear_positive", [0.177, 0.288]),
    esp.UniformPrior("self_recovery_rate_dict.smear_negative", [0.073, 0.209]),
    esp.UniformPrior("seed_time", [1800.0, 1950.0]),
    # esp.UniformPrior("seed_duration", [0.0, 1.0]),
    # esp.UniformPrior("self_rate", [1.0, 100.0]),
]
pop = pd.Series({2009: 1207100, 2019: 1194300})
notif = pd.Series({2011: 1495,2012: 1485,2013: 1369,2014:1405,2015:1642, 2016:1555, 2017:1440, 2018:1468, 2019:1417})
latent = pd.Series({2016:36})

targets = [
    est.NegativeBinomialTarget('total_population', pop, dispersion_param=2000),
    #est.NormalTarget('percentage_latent', latent, stdev=5)
]
calibration_model = BayesianCompartmentalModel(tb_model, params, priors, targets)

In [9]:
with pm.Model() as pmc_model:
    start_params = {k: np.clip(v, *calibration_model.priors[k].bounds(0.99)) for k, v in params.items() if k in calibration_model.priors}
    variables = epm.use_model(calibration_model)
    map_params = pm.find_MAP(start=start_params, vars=variables, include_transformed=False)
    map_params = {k: float(v) for k, v in map_params.items()}
    print('Best calibration parameters found:')
for i_param, param in enumerate(map_params):
    print(f'   {param}: {round_sigfig(map_params[param], 4)} (within bound {priors[i_param].bounds()}')


Best calibration parameters found:
   start_population_size: 14090.0 (within bound (1.0, 300000.0)
   contact_rate: 0.01999 (within bound (0.0001, 0.02)
   rr_infection_latent: 0.4766 (within bound (0.2, 0.5)
   rr_infection_recovered: 0.4995 (within bound (0.1, 0.5)
   smear_positive_death_rate: 0.4489 (within bound (0.335, 0.449)
   smear_negative_death_rate: 0.03499 (within bound (0.017, 0.035)
   smear_positive_self_recovery: 0.2878 (within bound (0.177, 0.288)
   smear_negative_self_recovery: 0.209 (within bound (0.073, 0.20900000000000002)
   progression_multiplier: 1.999 (within bound (0.1, 2.0)
   infect_death_rate_dict.smear_positive: 0.449 (within bound (0.335, 0.449)
   infect_death_rate_dict.smear_negative: 0.03499 (within bound (0.017, 0.035)
   self_recovery_rate_dict.smear_positive: 0.288 (within bound (0.177, 0.288)
   self_recovery_rate_dict.smear_negative: 0.2089 (within bound (0.073, 0.20900000000000002)
   seed_time: 1800.0 (within bound (1800.0, 1950.0)
   seed_du

In [10]:
map_params

{'start_population_size': 14086.68204080829,
 'contact_rate': 0.019990395820690225,
 'rr_infection_latent': 0.4766139262561174,
 'rr_infection_recovered': 0.49949083531643496,
 'smear_positive_death_rate': 0.44886040643658387,
 'smear_negative_death_rate': 0.034994648640436346,
 'smear_positive_self_recovery': 0.28784943979250416,
 'smear_negative_self_recovery': 0.20899970965449816,
 'progression_multiplier': 1.9992737932479652,
 'infect_death_rate_dict.smear_positive': 0.4489515882724915,
 'infect_death_rate_dict.smear_negative': 0.034992356043024975,
 'self_recovery_rate_dict.smear_positive': 0.28795286226532063,
 'self_recovery_rate_dict.smear_negative': 0.20894224565841085,
 'seed_time': 1800.4095099059316,
 'seed_duration': 0.9999978651066043,
 'self_rate': 99.95795823663732}

In [11]:
params.update(map_params)
tb_model.run(params)
derived_df_0 = tb_model.get_derived_outputs_df()
plots = {"total_population": {
      "title": "Population size",
      "output_key": "total_population",
      "times": [2009.0, 2019.0],
      "values": [1207100, 1194300],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
     "notifications": {
      "title": "Notifications",
      "output_key": "notifications",
      "times": [2011.0, 2012.0, 2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2018.0, 2019.0],
      "values": [1495, 1485, 1369, 1405, 1642, 1555, 1440, 1468, 1417],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
    "percentage_latent": {
      "title": "Percentage Latent",
      "output_key": "percentage_latent",
      "times": [2016.0],
      "values": [30.8],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
    
    }

In [12]:
fig2_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="total_population",
)
fig2_2 = px.scatter(x= plots['total_population']['times'], y = plots['total_population']['values'])
fig2_2.update_traces(marker=dict(color="red"))
fig2_3 = go.Figure(
    data=fig2_1.data + fig2_2.data,
)
fig2_3.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
)
fig2_3.show()


In [13]:
pop = derived_df_0[[f'total_populationXage_{i}' for i in age_strata]] 

In [14]:
pop.plot.area()

In [15]:
pop.plot.area(log_y=True)

In [16]:
derived_df_0

,infectious_population_size,latent_population_size,percentage_latent,prevalence_infectious,total_population,total_populationXage_0,total_populationXage_15,total_populationXage_35,total_populationXage_5,total_populationXage_50,total_populationXage_70
1800.0,0.0,0.0,0.0,0.0,1.408668e+04,2347.780340,2347.780340,2347.780340,2347.780340,2347.780340,2347.780340
1800.1,0.0,0.0,0.0,0.0,1.410205e+04,2348.252263,2357.716921,2341.334107,2370.108836,2346.278499,2338.356687
1800.2,0.0,0.0,0.0,0.0,1.411755e+04,2348.773811,2367.817529,2334.987652,2392.215287,2344.745093,2329.010194
1800.3,0.0,0.0,0.0,0.0,1.413319e+04,2349.344362,2378.079017,2328.741010,2414.102980,2343.181016,2319.740015
1800.4,0.0,0.0,0.0,0.0,1.414896e+04,2349.963308,2388.498296,2322.594200,2435.775149,2341.587153,2310.545312
...,...,...,...,...,...,...,...,...,...,...,...
2019.6,0.0,0.0,0.0,0.0,1.255207e+06,101232.118645,372690.228249,243348.194336,198473.802703,217528.267402,121933.944666
2019.7,0.0,0.0,0.0,0.0,1.256085e+06,101290.455176,372749.425213,243519.604941,198498.639076,217846.978516,122179.439899
2019.8,0.0,0.0,0.0,0.0,1.256962e+06,101349.083800,372808.569421,243690.122970,198524.390174,218164.922267,122424.831695
2019.9,0.0,0.0,0.0,0.0,1.257839e+06,101407.997577,372867.670248,243859.754367,198551.052586,218482.097325,122670.116940
